<a href="https://colab.research.google.com/github/JairsonAlbertoSami/desafio-junior-data-scientis/blob/main/Analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Técnico - Cientista de Dados Júnior - Rio de Janeiro

Este código foi desenvolvido como parte do desafio técnico para a posição de **Cientista de Dados Júnior** no **Rio de Janeiro**. O objetivo deste desafio é demonstrar habilidades em manipulação e análise de dados com python


In [76]:
# Instala BigQuery e basedosdados

!pip install google-cloud-bigquery
!pip install basedosdados

In [77]:
# importar as bibliotecas necessarias

import pandas as pd
import basedosdados as bd

In [78]:
# id do projeto no google cloud

project_id = "datariojairson"

In [79]:
# Autenticar no google cloude

from google.colab import auth
auth.authenticate_user()

In [80]:
# Consulta dados 'bairro' e armazena no DataFrame.

df_dados_mestres = bd.read_sql("SELECT* FROM `datario.dados_mestres.bairro`",
                 billing_project_id = project_id)

Downloading: 100%|██████████| 164/164 [00:01<00:00, 108.13rows/s]


In [81]:
# Consulta chamados do dia '2023-04-01' e armazena no DataFrame.
data_especifica = '2023-04-01'

consulta_sql = f"""
SELECT *
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '{data_especifica}'
"""

df_chamados = bd.read_sql(consulta_sql, billing_project_id=project_id)

Downloading: 100%|██████████| 1756/1756 [00:00<00:00, 3091.51rows/s]


# **Localização de chamados do 1746**

Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.




# 1. **Quantos chamados foram abertos no dia 01/04/2023?**

In [82]:
# Converte 'data_inicio' para datetime, filtra os chamados de 01/04/2023

df_chamados['data_inicio'] = pd.to_datetime(df_chamados['data_inicio'])
df_chamado_0104 = df_chamados[df_chamados['data_inicio'].dt.date == pd.to_datetime('2023-04-01').date()]
print(f"Foram abertos {len(df_chamado_0104)} chamados no dia 2023-04-01.")

Foram abertos 1756 chamados no dia 2023-04-01.


# 2. **Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?**

In [83]:
# Identifica e imprime o tipo de chamado mais frequente em 01/04/2023.

tipo_chamado_mais_frequente = df_chamado_0104['tipo'].value_counts().index[0]
print(f"O tipo de chamado com mais ocorrências em 01/04/2023 foi: {tipo_chamado_mais_frequente}, foram {df_chamado_0104['tipo'].value_counts()[0]} chamados")

O tipo de chamado com mais ocorrências em 01/04/2023 foi: Estacionamento irregular, foram 366 chamados


# 3. **Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?**

In [84]:
# Faz merge externo entre 'df_dados_mestres' e 'df_chamados'.
df_mestre_chamados_merged = pd.merge(df_dados_mestres, df_chamados, on='id_bairro', how='outer')
df_mestre_chamados_merged.head()

,id_bairro,nome,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,2,Gamboa,1,1.1,Centro,1,Portuaria,Centro,1.112903e+06,4612.833630,...,NaT,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaT
1,1,Saúde,1,1.1,Centro,1,Portuaria,Centro,3.638176e+05,2646.220568,...,NaT,<NA>,H,F,No prazo,Encerrado,Não constatado,None,0,2023-04-01
2,4,Caju,1,1.1,Centro,1,Portuaria,Centro,5.347481e+06,19800.522524,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Atendido,None,0,2023-04-01
3,4,Caju,1,1.1,Centro,1,Portuaria,Centro,5.347481e+06,19800.522524,...,NaT,<NA>,D,F,No prazo,Encerrado,Não atendido,None,0,2023-04-01
4,3,Santo Cristo,1,1.1,Centro,1,Portuaria,Centro,1.684721e+06,6743.227885,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Atendido,None,2,2023-04-01


In [85]:
# Exibe os 3 bairros com mais chamados em 01/04/2023.

top_3_bairros_merged = df_mestre_chamados_merged['nome'].value_counts().nlargest(3)
print(f"Os 3 bairros com mais chamados abertos em 01/04/2023 foram: {', '.join(top_3_bairros_merged.index)}")

Os 3 bairros com mais chamados abertos em 01/04/2023 foram: Campo Grande, Tijuca, Barra da Tijuca


# 4. **Qual o nome da subprefeitura com mais chamados abertos nesse dia?**

In [86]:
# Identifica a subprefeitura com mais chamados em 01/04/2023 e exibe.

top_subprefeitura = df_mestre_chamados_merged['subprefeitura'].value_counts().nlargest(1)
top_subprefeitura = df_mestre_chamados_merged['subprefeitura'].value_counts().index[0]
print(f"A subprefeitura com mais chamados abertos em 01/04/2023 foi: {top_subprefeitura}, com {df_mestre_chamados_merged['subprefeitura'].value_counts()[0]} chamados")


A subprefeitura com mais chamados abertos em 01/04/2023 foi: Zona Norte, com 515 chamados


# 5. **Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?**

In [87]:
# Identifica chamados sem bairro associado e fornece possíveis explicações.
chamados_sem_bairro = df_mestre_chamados_merged[df_mestre_chamados_merged['nome'].isnull()]
num_chamados_sem_bairro = chamados_sem_bairro.shape[0]

if num_chamados_sem_bairro > 0:
    print(f"OK! Sim, existem {num_chamados_sem_bairro} chamados sem bairro associado.")
    print("A ausência de informações sobre o bairro pode ocorrer por várias razões. Aqui estão alguns possíveis motivos:")
    print("1. Erro na entrada de dados: O campo correspondente ao bairro pode não ter sido preenchido adequadamente no momento do registro do chamado.")
    print("2. Chamados de locais fora da divisão oficial de bairros: Alguns chamados podem estar relacionados a áreas que não são formalmente divididas em bairros, como estradas, áreas rurais ou zonas de transição.")
    print("3. Base de dados incompleta ou desatualizada: A lista de bairros pode não estar completa ou pode não ter sido atualizada recentemente, o que pode resultar na ausência de informações correspondentes para determinados chamados.")
else:
    print("Não, todos os chamados abertos nesse dia estão associados a um bairro.")

OK! Sim, existem 73 chamados sem bairro associado.
A ausência de informações sobre o bairro pode ocorrer por várias razões. Aqui estão alguns possíveis motivos:
1. Erro na entrada de dados: O campo correspondente ao bairro pode não ter sido preenchido adequadamente no momento do registro do chamado.
2. Chamados de locais fora da divisão oficial de bairros: Alguns chamados podem estar relacionados a áreas que não são formalmente divididas em bairros, como estradas, áreas rurais ou zonas de transição.
3. Base de dados incompleta ou desatualizada: A lista de bairros pode não estar completa ou pode não ter sido atualizada recentemente, o que pode resultar na ausência de informações correspondentes para determinados chamados.


In [88]:
# Identifica chamados sem subprefeitura associado e fornece possíveis explicações.

df_dados_mestres_sem_subprefei= df_mestre_chamados_merged[df_mestre_chamados_merged['subprefeitura'].isnull()]
num_df_dados_mestres_sem_subprefei = df_dados_mestres_sem_subprefei.shape[0]

if num_df_dados_mestres_sem_subprefei >0:
    print(f"Sim, existem {num_df_dados_mestres_sem_subprefei} chamados sem subprefeitura associado.")
    print("A ausência de informações sobre o subprefeitura pode ocorrer por várias razões. Aqui estão alguns possíveis motivos:")
    print("1. Erro na entrada de dados:** O campo correspondente ao subprefeitura pode não ter sido preenchido adequadamente no momento do registro do chamado.")
    print("2. Chamados de locais fora da divisão oficial de subprefeitura:** Alguns chamados podem estar relacionados a áreas que não são formalmente divididas em subprefeituras, como estradas, áreas rurais ou zonas de transição.")
    print("3. Base de dados incompleta ou desatualizada: A lista de subprefeitura pode não estar completa ou pode não ter sido atualizada recentemente, o que pode resultar na ausência de informações correspondentes para determinados chamados.")
else:
  print("Não, todos os chamados abertos nesse dia estão associados a uma subprefeitura.")


Sim, existem 73 chamados sem subprefeitura associado.
A ausência de informações sobre o subprefeitura pode ocorrer por várias razões. Aqui estão alguns possíveis motivos:
1. Erro na entrada de dados:** O campo correspondente ao subprefeitura pode não ter sido preenchido adequadamente no momento do registro do chamado.
2. Chamados de locais fora da divisão oficial de subprefeitura:** Alguns chamados podem estar relacionados a áreas que não são formalmente divididas em subprefeituras, como estradas, áreas rurais ou zonas de transição.
3. Base de dados incompleta ou desatualizada: A lista de subprefeitura pode não estar completa ou pode não ter sido atualizada recentemente, o que pode resultar na ausência de informações correspondentes para determinados chamados.


# **Chamados do 1746 em grandes eventos**

Utilize a tabela de Chamados do 1746 e a tabela de Ocupação Hoteleira em Grandes Eventos no Rio para as perguntas de 6-10. Para todas as perguntas considere o subtipo de chamado "Perturbação do sossego


In [169]:
# Consulta chamados entre '2022-01-01' e '2023-12-31' e armazena no DataFrame.

data_fim = '2023-12-31'
data_inicio = '2022-01-01'

consulta_sql = f"""
SELECT *
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) BETWEEN '{data_inicio}' AND '{data_fim}'
"""

df_chamados_grandes_eventos = bd.read_sql(consulta_sql, billing_project_id=project_id)

Downloading: 100%|██████████| 1634336/1634336 [06:57<00:00, 3914.96rows/s]


In [170]:
df_chamados_grandes_eventos.head(2)

,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,17556781,11,2022-08-04 00:58:17,2022-09-22 17:47:21,None,None,None,<NA>,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Não atendido,None,0,2022-08-01
1,17561271,1,2022-08-04 21:32:27,2022-08-05 23:43:00,6,1,157610,152,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2022-08-01


# 6. **Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?**

In [171]:
# Filtra chamados relacionados à 'Perturbação do sossego' e conta os registros entre 2022 e 2023.

df_pertubacao_sossego = df_chamados_grandes_eventos[df_chamados_grandes_eventos["subtipo"].str.contains("Perturbação do sossego")]

inicio = pd.to_datetime('2022-01-01')
fim = pd.to_datetime('2023-12-31')

df_filtrado = df_pertubacao_sossego[(df_pertubacao_sossego['data_inicio'] >= inicio) & (df_pertubacao_sossego['data_inicio'] <= fim)]

num_chamados = df_filtrado.shape[0]

print(f"Entre o começo de 2022 e o final de 2023, foram registrados {num_chamados} chamados relacionados à 'Perturbação do sossego'.")


Entre o começo de 2022 e o final de 2023, foram registrados 42830 chamados relacionados à 'Perturbação do sossego'.


# 7. **Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).**

In [176]:
# Lê dados sobre eventos de ocupação hoteleira e converte as colunas de datas para datetime.

df_eventos = bd.read_sql("SELECT* FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`",
                 billing_project_id = project_id)

Downloading: 100%|██████████| 8/8 [00:00<00:00, 37.81rows/s]


In [177]:
df_eventos.drop_duplicates(inplace=True)
df_eventos

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
2,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
4,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
6,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251


In [178]:
df_eventos['data_inicial'] = pd.to_datetime(df_eventos['data_inicial'])
df_eventos['data_final'] = pd.to_datetime(df_eventos['data_final'])

In [175]:
# df_eventos.loc[df_eventos.index[1], 'evento'] = 'Rock in Rio - A'
# df_eventos.loc[df_eventos.index[2], 'evento'] = 'Rock in Rio - B'

In [179]:
def encontrar_evento(row):
    for _, evento in df_eventos.iterrows():
        if evento['data_inicial'] <= row['data_inicio'] <= evento['data_final']:
            return evento['evento']
        if pd.notna(row['data_fim']) and evento['data_inicial'] <= row['data_fim'] <= evento['data_final']:
            return evento['evento']
    return None

df_pertubacao_sossego['evento'] = df_pertubacao_sossego.apply(encontrar_evento, axis=1)

In [180]:
eventos_desejados = ['Carnaval', 'Reveillon', 'Rock in Rio']

In [181]:
df_chamados_eventos = df_pertubacao_sossego[df_pertubacao_sossego['evento'].isin(eventos_desejados)]

print(f'Foram no total {len(df_chamados_eventos)} chamados nos eventos: Carnaval, Reveillon, Rock in Rio')

Foram no total 1938 chamados nos eventos: Carnaval, Reveillon, Rock in Rio


# 8. **Quantos chamados desse subtipo foram abertos em cada evento?**

In [182]:
# Conta e imprime o número de chamados por evento específico.

contagem_chamados_por_evento = df_chamados_eventos['evento'].value_counts()
print("Número de chamados por evento:")
print(' ')
print(contagem_chamados_por_evento)


Número de chamados por evento:
 
Rock in Rio    1491
Reveillon       231
Carnaval        216
Name: evento, dtype: int64


# 9. **Qual evento teve a maior média diária de chamados abertos desse subtipo?**

In [183]:
# Calcula a duração dos eventos e a média diáia de chamados por evento.
df_eventos['duracao'] = (df_eventos['data_final'] - df_eventos['data_inicial']).dt.days + 1

# Calcula a média diária de chnados por evento.
media_diaria_por_evento = df_chamados_eventos['evento'].value_counts() / df_eventos.set_index('evento')['duracao']

In [184]:
df_chamados_eventos['evento'].value_counts()

,evento
Rock in Rio,1491
Reveillon,231
Carnaval,216


In [185]:
evento_maior_media = media_diaria_por_evento.idxmax()
maior_media = media_diaria_por_evento.max()

print(f"O evento com a maior média diária de chmados 'Perturbação do Sossego' fpi {evento_maior_media} com {maior_media:.2f} chamados por dia.")

O evento com a maior média diária de chmados 'Perturbação do Sossego' fpi Rock in Rio com 497.00 chamados por dia.


# 10. **Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.**

In [186]:
inicio_periodo = pd.to_datetime('2022-01-01')
fim_periodo = pd.to_datetime('2023-12-31')
periodo_total_dias = (fim_periodo - inicio_periodo).days
total_chamados = df_pertubacao_sossego.shape[0]
media_diaria_total = total_chamados / periodo_total_dias

In [187]:
medias_diarias = pd.DataFrame(index=eventos_desejados + ['Total'], columns=['Media Diaria'])

for evento in eventos_desejados:
    duracao_evento = df_eventos[df_eventos['evento'] == evento]['duracao'].values[0]
    num_chamados_evento = df_chamados_eventos[df_chamados_eventos['evento'] == evento].shape[0]
    medias_diarias.loc[evento, 'Media Diaria'] = num_chamados_evento / duracao_evento

medias_diarias.loc['Total', 'Media Diaria'] = media_diaria_total

In [188]:
for evento in eventos_desejados:
    media_evento = medias_diarias.loc[evento, 'Media Diaria']
    comparacao = "maior" if media_evento > media_diaria_total else "menor"
    diferenca_percentual = abs(media_evento - media_diaria_total) / media_diaria_total * 100
    print(f"A média diária de chamados durante o {evento} foi {diferenca_percentual:.2f}% {comparacao} do que a média diária total.")

A média diária de chamados durante o Carnaval foi 8.09% menor do que a média diária total.
A média diária de chamados durante o Reveillon foi 31.06% maior do que a média diária total.
A média diária de chamados durante o Rock in Rio foi 745.93% maior do que a média diária total.
